In [1]:
# This Helper fetches de resolver dataset from the EBI identifiers.org platform, to transform it and put it into
# the new MongoDB based data backend.
# Author: Manuel Bernal Llinares <mbdebian@gmail.com>